In [9]:
import pandas as pd
import time
from tqdm import tqdm
from uuid import uuid4
from langchain_pinecone import PineconeVectorStore

from pinecone import Pinecone, ServerlessSpec
from langchain_openai import OpenAIEmbeddings
import json
from typing import List

import os
from dotenv import load_dotenv
load_dotenv()

True

In [15]:
INDEX_NAME = "funeral-services"
pc = Pinecone()
index = pc.Index(INDEX_NAME)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
print(index.describe_index_stats(), '\n\n')

{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'digital_legacy': {'vector_count': 68},
                'funeral_facilities': {'vector_count': 2518},
                'guide': {'vector_count': 4},
                'ordinance': {'vector_count': 234}},
 'total_vector_count': 2824,
 'vector_type': 'dense'} 




In [ ]:
# index.delete(delete_all=True, namespace="funeral_facilities")
# print(index.describe_index_stats(), '\n\n')

In [3]:
CHUNK_FILES = [
    "identity_verification_service_chunked.json",
    "naver_data1_chunked.json",
    "naver_data2_chunked.json",
    "online_shoppingmal_chunked.json",
    "google_data1_chunked.json",
    "google_data2_chunked.json",
    "kakaotalk_data1_chunked.json",
    "kakaotalk_data2_chunked.json"
]

In [4]:
def load_chunks(path: str):
    with open(path, "r", encoding="utf-8") as f:
        chunks = json.load(f)
    print(f"[INFO] 총 {len(chunks)}개의 청크 로드 완료.")
    return chunks

In [ ]:
def upsert_chunks_to_pinecone(chunks, batch_size: int = 100):
    total = len(chunks)
    vectorstore = PineconeVectorStore(
            index_name=INDEX_NAME,
            embedding=embeddings,
            namespace='digital_legacy'
    )
    for start in range(0, total, batch_size):
        end = min(start + batch_size, total)
        batch = chunks[start:end]

        # 1) 텍스트 / ID 추출
        texts = [c["text"] for c in batch]   # JSON의 text 필드
        ids = [c["id"] for c in batch]       # JSON의 id 필드

        # 2) 메타데이터 구성
        metadatas = []
        for c in batch:
            base_meta = c.get("metadata", {}) or {}

            # 기본 메타데이터 + 원본 metadata 합치기
            meta = {
                "chunk_id": c["id"],
                "text": c["text"],
            }
            meta.update(base_meta)

            # 🔥 None 제거
            clean_meta = {k: v for k, v in meta.items() if v is not None}

            metadatas.append(clean_meta)

        vectorstore.add_texts(
            texts=texts,
            metadatas=metadatas,
            ids=ids
        )    
        
        print(f"[INFO] 업서트 진행: {end}/{total} 개 완료.")

In [6]:
def upload_chunk_file(json_path: str, source_name: str | None = None):
    """
    json_path에 있는 청킹 JSON 파일을 로드해서
    Pinecone 인덱스로 업로드하는 편의 함수.
    """
    print(f"[INFO] 업로드 시작: {json_path}")

    if not os.path.exists(json_path):
        print(f"[WARN] 파일 없음, 건너뜀: {json_path}")
        return

    chunks = load_chunks(json_path)

    # 필요하다면 여기서 source_name을 메타데이터에 추가하는 로직을
    # upsert_chunks_to_pinecone 쪽으로 넘겨도 됨.
    upsert_chunks_to_pinecone(chunks, batch_size=100)

    print(f"[INFO] 업로드 완료: {json_path}")

In [10]:
BASE_DIR = '../data/processed'

for filename in CHUNK_FILES:
    json_path = os.path.join(BASE_DIR, filename)
    upload_chunk_file(json_path)

# 루프 끝난 뒤, 인덱스 통계 확인
stats = index.describe_index_stats()
print("[INFO] 인덱스 통계:")
print(stats)

[INFO] 업로드 시작: ../data/processed\identity_verification_service_chunked.json
[INFO] 총 5개의 청크 로드 완료.
[INFO] 업서트 진행: 5/5 개 완료.
[INFO] 업로드 완료: ../data/processed\identity_verification_service_chunked.json
[INFO] 업로드 시작: ../data/processed\naver_data1_chunked.json
[INFO] 총 6개의 청크 로드 완료.
[INFO] 업서트 진행: 6/6 개 완료.
[INFO] 업로드 완료: ../data/processed\naver_data1_chunked.json
[INFO] 업로드 시작: ../data/processed\naver_data2_chunked.json
[INFO] 총 11개의 청크 로드 완료.
[INFO] 업서트 진행: 11/11 개 완료.
[INFO] 업로드 완료: ../data/processed\naver_data2_chunked.json
[INFO] 업로드 시작: ../data/processed\online_shoppingmal_chunked.json
[INFO] 총 12개의 청크 로드 완료.
[INFO] 업서트 진행: 12/12 개 완료.
[INFO] 업로드 완료: ../data/processed\online_shoppingmal_chunked.json
[INFO] 업로드 시작: ../data/processed\google_data1_chunked.json
[INFO] 총 7개의 청크 로드 완료.
[INFO] 업서트 진행: 7/7 개 완료.
[INFO] 업로드 완료: ../data/processed\google_data1_chunked.json
[INFO] 업로드 시작: ../data/processed\google_data2_chunked.json
[INFO] 총 10개의 청크 로드 완료.
[INFO] 업서트 진행: 10/10 개 완료.
[INFO] 업로드 완